# Loading the dataset beans from tensorflow

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Step 1: Install TensorFlow Datasets if not already installed
# !pip install tensorflow-datasets

# Step 2: Import Libraries
import tensorflow as tf
import tensorflow_datasets as tfds

# Step 3: Load the Dataset
dataset_name = "beans"
(train_ds, test_ds,valid), ds_info = tfds.load(
    dataset_name,
    split=["train", "test" , "validation"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
# Print the number of samples in the train and test datasets
print("Number of samples in train dataset:", ds_info.splits["train"].num_examples)
print("Number of samples in test dataset:", ds_info.splits["test"].num_examples)
print("Number of samples in validation dataset:", ds_info.splits["validation"].num_examples)



Number of samples in train dataset: 1034
Number of samples in test dataset: 128
Number of samples in test dataset: 133


#Getting the input shape

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the dataset
dataset_name = "beans"
(train_ds, _), _ = tfds.load(
    dataset_name,
    split=["train", "test" , "validation"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Take the first batch from the dataset
for image_batch, label_batch in train_ds.take(1):
    input_shape = image_batch.shape[1:]  # Extract input shape from the image batch
    print("Input shape:", input_shape)


Input shape: (500, 3)


#Testing using the validation dataset

In [ ]:
# Step 1: Install TensorFlow Datasets if not already installed
# !pip install tensorflow-datasets
8
# Step 2: Import Libraries
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Step 3: Load the Dataset
dataset_name = "beans"
(train_ds, test_ds,val), ds_info = tfds.load(
    dataset_name,
    split=["train", "test","validation"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
# Print the number of samples in the train and test datasets
print("Number of samples in train dataset:", ds_info.splits["train"].num_examples)
print("Number of samples in test dataset:", ds_info.splits["test"].num_examples)
print("Number of samples in validation dataset:", ds_info.splits["validation"].num_examples)
# Step 4: Preprocess the Data
def preprocess_image(image, label):
    image = tf.image.resize(image, [32, 32])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(preprocess_image)
test_ds = test_ds.map(preprocess_image)
val = val.map(preprocess_image)

# Step 5: Batch and Prefetch
BATCH_SIZE = 128
train_ds = train_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)
val= val.batch(BATCH_SIZE)


train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)
val =val.prefetch(tf.data.experimental.AUTOTUNE)
# Define data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
])

# Step 6: Model Training
model = Sequential([
    data_augmentation,
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(500, 500, 3)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define early stopping callback
#early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with data augmentation and early stopping
history = model.fit(train_ds, epochs=70, validation_data=val)

# Evaluate the model
loss, accuracy = model.evaluate(val)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
######################################### 0.82 best accuracy, decrease batches128 and increase epoches50?  add a set of conv2d also

Number of samples in train dataset: 1034
Number of samples in test dataset: 128
Number of samples in validation dataset: 133
Epoch 1/70
9/9 [==============================] - 11s 806ms/step - loss: 1.4673 - accuracy: 0.3114 - val_loss: 1.3939 - val_accuracy: 0.3383
Epoch 2/70
9/9 [==============================] - 8s 754ms/step - loss: 1.2084 - accuracy: 0.3588 - val_loss: 1.1217 - val_accuracy: 0.3308
Epoch 3/70
9/9 [==============================] - 8s 882ms/step - loss: 1.1205 - accuracy: 0.3627 - val_loss: 1.0839 - val_accuracy: 0.3383
Epoch 4/70
9/9 [==============================] - 8s 826ms/step - loss: 1.0871 - accuracy: 0.4043 - val_loss: 1.0830 - val_accuracy: 0.3910
Epoch 5/70
9/9 [==============================] - 8s 740ms/step - loss: 1.0716 - accuracy: 0.4352 - val_loss: 1.0559 - val_accuracy: 0.5113
Epoch 6/70
9/9 [==============================] - 8s 851ms/step - loss: 1.0323 - accuracy: 0.5068 - val_loss: 0.9308 - val_accuracy: 0.5865
Epoch 7/70
9/9 [==================

#Testing using the test dataset

In [ ]:

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Load the Dataset
dataset_name = "beans"
(train_ds, test_ds), ds_info = tfds.load(
    dataset_name,
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
# Print the number of samples in the train and test datasets
print("Number of samples in train dataset:", ds_info.splits["train"].num_examples)
print("Number of samples in test dataset:", ds_info.splits["test"].num_examples)

# Preprocess the Data
def preprocess_image(image, label):
    image = tf.image.resize(image, [32, 32])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(preprocess_image)
test_ds = test_ds.map(preprocess_image)

# Batch and Prefetch
BATCH_SIZE = 128
train_ds = train_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)


train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)

# Define data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
])

# Step 6: Model Training
model = Sequential([
    data_augmentation,
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(500, 500, 3)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy', #classification
              metrics=['accuracy'])

#Stochastic gradient descent is an iterative learning algorithm that uses a training dataset to update a model.
#The batch size is a hyperparameter of gradient descent that controls the number of training samples to work through before the model’s internal parameters are updated.
#The number of epochs is a hyperparameter of gradient descent that controls the number of complete passes through the training dataset.

# Train the model with data augmentation and early stopping
history = model.fit(train_ds, epochs=100, validation_data=test_ds)

# Evaluate the model
loss, accuracy = model.evaluate(test_ds)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
######################################### 0.82 best accuracy, decrease batches128 and increase epoches50?  add a set of conv2d also

Number of samples in train dataset: 1034
Number of samples in test dataset: 128
Epoch 1/100
9/9 [==============================] - 13s 1s/step - loss: 1.4667 - accuracy: 0.3337 - val_loss: 1.3260 - val_accuracy: 0.3359
Epoch 2/100
9/9 [==============================] - 7s 732ms/step - loss: 1.2534 - accuracy: 0.3182 - val_loss: 1.1236 - val_accuracy: 0.3281
Epoch 3/100
9/9 [==============================] - 9s 854ms/step - loss: 1.1112 - accuracy: 0.3704 - val_loss: 1.0285 - val_accuracy: 0.4375
Epoch 4/100
9/9 [==============================] - 9s 906ms/step - loss: 1.0562 - accuracy: 0.4516 - val_loss: 1.0354 - val_accuracy: 0.3984
Epoch 5/100
9/9 [==============================] - 8s 865ms/step - loss: 1.0290 - accuracy: 0.5000 - val_loss: 0.9495 - val_accuracy: 0.5312
Epoch 6/100
9/9 [==============================] - 9s 765ms/step - loss: 0.9496 - accuracy: 0.5551 - val_loss: 0.8423 - val_accuracy: 0.6484
Epoch 7/100
9/9 [==============================] - 9s 883ms/step - loss: 0.8

#Adding vgg16 ti increase accuracy

In [ ]:
# Step 1: Install TensorFlow Datasets if not already installed
# !pip install tensorflow-datasets

# Step 2: Import Libraries
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

# Step 3: Load the Dataset
dataset_name = "beans"
(train_ds, test_ds), ds_info = tfds.load(
    dataset_name,
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
# Print the number of samples in the train and test datasets
print("Number of samples in train dataset:", ds_info.splits["train"].num_examples)
print("Number of samples in test dataset:", ds_info.splits["test"].num_examples)

# Step 4: Preprocess the Data
def preprocess_image(image, label):
    image = tf.image.resize(image, [224, 224])  # VGG16 input size
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(preprocess_image)
test_ds = test_ds.map(preprocess_image)

# Step 5: Batch and Prefetch
BATCH_SIZE = 32  # Reduce batch size for VGG16
train_ds = train_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)

# Step 6: Load VGG16 base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers: #prevent overfitting
    layer.trainable = False

# Step 7: Model Construction
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Step 8: Model Training
history = model.fit(train_ds, epochs=10, validation_data=test_ds)

# Step 9: Model Evaluation
loss, accuracy = model.evaluate(test_ds)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Number of samples in train dataset: 1034
Number of samples in test dataset: 128
Epoch 1/10
33/33 [==============================] - 697s 21s/step - loss: 4.0062 - accuracy: 0.5387 - val_loss: 1.5492 - val_accuracy: 0.6797
Epoch 2/10
33/33 [==============================] - 672s 20s/step - loss: 0.7580 - accuracy: 0.7669 - val_loss: 0.4570 - val_accuracy: 0.8281
Epoch 3/10
33/33 [==============================] - 666s 20s/step - loss: 0.3820 - accuracy: 0.8482 - val_loss: 0.4396 - val_accuracy: 0.8281
Epoch 4/10
33/33 [==============================] - 678s 21s/step - loss: 0.3087 - accuracy: 0.8791 - val_loss: 0.5192 - val_accuracy: 0.7656
Epoch 5/10
33/33 [==============================] - 682s 21s/step - loss: 0.2317 - accuracy: 0.9188 - val_loss: 0.4095 - val_accuracy: 0.8359
Epoch 6/10
33/33 [==============================] - 697s 21s/step - loss: 0.1661 - accuracy: 0.9410 - val_loss: 0.4201 - val_accuracy: 0.7969
Epoch 7/10
33/33 [==============================] - 695s 21s/step - 

In [ ]:
# Step 1: Install TensorFlow Datasets if not already installed
# !pip install tensorflow-datasets

# Step 2: Import Libraries
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

# Step 3: Load the Dataset
dataset_name = "beans"
(train_ds, test_ds), ds_info = tfds.load(
    dataset_name,
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
# Print the number of samples in the train and test datasets
print("Number of samples in train dataset:", ds_info.splits["train"].num_examples)
print("Number of samples in test dataset:", ds_info.splits["test"].num_examples)

# Step 4: Preprocess the Data
def preprocess_image(image, label):
    image = tf.image.resize(image, [224, 224])  # VGG16 input size
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(preprocess_image)
test_ds = test_ds.map(preprocess_image)

# Step 5: Batch and Prefetch
BATCH_SIZE = 32  # Reduce batch size for VGG16
train_ds = train_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)

# Step 6: Load VGG16 base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Step 7: Model Construction
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Step 8: Model Training

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Step 9: Model Training with Early Stopping
history = model.fit(train_ds, epochs=50, validation_data=test_ds, callbacks=[early_stopping])

# Step 9: Model Evaluation
loss, accuracy = model.evaluate(test_ds)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1034 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/beans/0.1.0.incompleteLCRF9N/beans-train.tfrecord*...:   0%|          | 0/…

Generating validation examples...:   0%|          | 0/133 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/beans/0.1.0.incompleteLCRF9N/beans-validation.tfrecord*...:   0%|         …

Generating test examples...:   0%|          | 0/128 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/beans/0.1.0.incompleteLCRF9N/beans-test.tfrecord*...:   0%|          | 0/1…

Dataset beans downloaded and prepared to /root/tensorflow_datasets/beans/0.1.0. Subsequent calls will reuse this data.
Number of samples in train dataset: 1034
Number of samples in test dataset: 128
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/50
33/33 [==============================] - 632s 19s/step - loss: 3.1532 - accuracy: 0.5735 - val_loss: 0.8652 - val_accuracy: 0.7188
Epoch 2/50
33/33 [==============================] - 612s 19s/step - loss: 0.6595 - accuracy: 0.7756 - val_loss: 0.3979 - val_accuracy: 0.8203
Epoch 3/50
33/33 [==============================] - 614s 19s/step - loss: 0.3100 - accuracy: 0.8743 - val_loss: 0.4025 - val_accuracy: 0.8359
Epoch 4/50
33/33 [==============================] - 623s 19s/step - loss: 0.2178 - accuracy: 0.9188 - val_loss: 0.6445 - val_accuracy: 0.7578
Epoch 5/50
4/4 [==============================] - 66s 17s/step - loss: 0.3979 - accuracy: 0.8203
Test Loss: 0.39794132113456726
Test Accuracy: 0.8203125


In [ ]:
# Step 1: Install TensorFlow Datasets if not already installed
# !pip install tensorflow-datasets

# Step 2: Import Libraries
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

# Step 3: Load the Dataset
dataset_name = "beans"
(train_ds, test_ds), ds_info = tfds.load(
    dataset_name,
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
# Print the number of samples in the train and test datasets
print("Number of samples in train dataset:", ds_info.splits["train"].num_examples)
print("Number of samples in test dataset:", ds_info.splits["test"].num_examples)

# Step 4: Preprocess the Data
def preprocess_image(image, label):
    image = tf.image.resize(image, [224, 224])  # VGG16 input size
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(preprocess_image)
test_ds = test_ds.map(preprocess_image)

# Step 5: Batch and Prefetch
BATCH_SIZE = 32  # Reduce batch size for VGG16
train_ds = train_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)

# Step 6: Load VGG16 base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Step 7: Model Construction
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Step 8: Model Training

early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

# Step 9: Model Training with Early Stopping
history = model.fit(train_ds, epochs=50, validation_data=test_ds, callbacks=[early_stopping])

# Step 9: Model Evaluation
loss, accuracy = model.evaluate(test_ds)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Number of samples in train dataset: 1034
Number of samples in test dataset: 128
Epoch 1/50
33/33 [==============================] - 626s 19s/step - loss: 4.4007 - accuracy: 0.4913 - val_loss: 0.7394 - val_accuracy: 0.7422
Epoch 2/50
33/33 [==============================] - 623s 19s/step - loss: 0.8697 - accuracy: 0.7495 - val_loss: 0.4582 - val_accuracy: 0.7656
Epoch 3/50
33/33 [==============================] - 607s 18s/step - loss: 0.3879 - accuracy: 0.8511 - val_loss: 0.5103 - val_accuracy: 0.7891
Epoch 4/50
33/33 [==============================] - 621s 19s/step - loss: 0.3260 - accuracy: 0.8607 - val_loss: 0.4229 - val_accuracy: 0.8359
Epoch 5/50
33/33 [==============================] - 622s 19s/step - loss: 0.2228 - accuracy: 0.9149 - val_loss: 0.4189 - val_accuracy: 0.8047
Epoch 6/50
33/33 [==============================] - 623s 19s/step - loss: 0.1368 - accuracy: 0.9545 - val_loss: 0.5242 - val_accuracy: 0.7891
Epoch 7/50
33/33 [==============================] - 606s 18s/step - 